<a href="https://colab.research.google.com/github/mijanr/TimeSeries/blob/master/Ford_data_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
#1d interpolation
from scipy.interpolate import interp1d

#tensorboard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/ford_data_classification')

In [31]:
root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

In [32]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)

In [33]:
x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

In [34]:
#convert y to 0 and 1
y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

In [35]:
#normalize x
x_train = MinMaxScaler().fit_transform(x_train)
x_test = MinMaxScaler().fit_transform(x_test)

In [36]:
#interpolate x
length = 100
X_train_xgb = np.zeros((x_train.shape[0], length))
X_test_xgb = np.zeros((x_test.shape[0], length))
for i in range(x_train.shape[0]):
    f = interp1d(np.arange(x_train.shape[1]), x_train[i])
    X_train_xgb[i] = f(np.linspace(0, x_train.shape[1]-1, length))
for i in range(x_test.shape[0]):
    f = interp1d(np.arange(x_test.shape[1]), x_test[i])
    X_test_xgb[i] = f(np.linspace(0, x_test.shape[1]-1, length))

In [37]:
X_train_xgb.shape   

(3601, 100)

In [38]:
#use Xgboost classifier
model = XGBClassifier(max_depth=10, n_estimators=1000, learning_rate=0.1)
model.fit(X_train_xgb, y_train)

XGBClassifier(max_depth=10, n_estimators=1000)

In [39]:
#make predictions for test data
y_pred = model.predict(X_test_xgb)
#evaluate predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 82.35%


In [40]:
#expand dimension
x_train = np.expand_dims(x_train, axis=1)
x_test = np.expand_dims(x_test, axis=1)

In [41]:
#check shapes
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(3601, 1, 500) (3601,) (1320, 1, 500) (1320,)


In [42]:
#convert to torch tensors
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()

In [43]:
#convert to torch dataset
train = TensorDataset(x_train, y_train)
test = TensorDataset(x_test, y_test)
#convert to dataloader
train_loader = DataLoader(train, batch_size=64, shuffle=True)
test_loader = DataLoader(test, batch_size=64, shuffle=False)

In [44]:
#check shapes
for x, y in train_loader:
    print(x.shape, y.shape)
    break

torch.Size([64, 1, 500]) torch.Size([64])


In [53]:
#1D CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.sequential_1 = nn.Sequential(
            nn.Conv1d(1, 64, 3),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 3),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(128, 128, 3),
            nn.ReLU(),
            #global average pooling
            nn.AdaptiveAvgPool1d(1)
        )
        self.sequential_2 = nn.Sequential(
            nn.LazyLinear(out_features=128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )
        
    def forward(self, x):
        x = self.sequential_1(x)
        x = x.view(x.size(0), -1)
        x = self.sequential_2(x)
        return x

In [54]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = CNN().to(device)

cuda


In [55]:
#feed a sample to the model
for x, y in train_loader:
    x = x.to(device)
    print(model(x).shape)
    break

torch.Size([64, 2])


In [56]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [57]:
#training
n_epochs = 500
for epoch in range(n_epochs):
    train_loss = 0.0
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)
    train_loss = train_loss / len(train_loader.dataset)
    if epoch % 100 == 0:
        print("Epoch: {} \tTraining Loss: {:.6f}".format(epoch, train_loss))
    #log loss
    writer.add_scalar("training loss", train_loss, epoch)
    #log accuracy
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)
            outputs = model(x)
            _, predicted = torch.max(outputs.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
    accuracy = 100 * correct / total
    writer.add_scalar("accuracy", accuracy, epoch)
    #close writer
    writer.close()

Epoch: 0 	Training Loss: 0.693797
Epoch: 100 	Training Loss: 0.211424
Epoch: 200 	Training Loss: 0.170444
Epoch: 300 	Training Loss: 0.125387
Epoch: 400 	Training Loss: 0.065159


In [58]:
#check the mdodel on test data
with torch.no_grad():
    correct = 0
    total = 0
    for x, y in test_loader:
        x = x.to(device)
        y = y.to(device)
        outputs = model(x)
        _, predicted = torch.max(outputs.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()
    print(f'Accuracy: {100 * correct / total} %')

Accuracy: 86.89393939393939 %
